In [53]:
def only_move(data,imp,num_PLAY,price,dec):
    import math
    #自分のIDが2ならここで入れ替える
    #その場合agentIDにnum_agentを+
    num_agent=len(data["teams"][0]["agents"])#エージェントの数
    exe_turn=math.floor(24/num_agent)
    nodes=[]
    acts=[]
    send_acts=[]
    for i in range(num_agent):
        only_move_AI(imp,num_PLAY,data,price,i,num_agent,nodes,exe_turn)
        if dec == "S":
            n=get_max_sel(nodes)
            s=get_max_sel(nodes[n]["child"])
            acts.append(n+1)
        elif dec == "U":
            n=get_max_ucb()
            acts.append(n)
            s=get_max_ucb()
        nodes=nodes[n]["child"][s]["child"]
    for i in range(num_agent):
        act=get_act()
        act["agentID"]=i+1
        act["type"]="move"
        m_y=math.floor(acts[i]/3)-1
        act["dy"]=m_y
        act["dx"]=(acts[i]%3)-1
        send_acts.append(act)
    return send_acts

In [55]:
def main(imp,num_PLAY,price,dec):#decは選択数依存ならS  UCB依存ならU
    import json
    f=open('A-1.json')
    data=json.load(f)
    print(data)
    send_acts=only_move(data,imp,num_PLAY,price,dec)
    print(send_acts)
    print(data)
if __name__=="__main__":
    from time import time
    t1=time()
    main(22,10000,10,"S")
    t2=time()
    print(t2-t1)

{'width': 10, 'height': 10, 'points': [[12, 3, 5, 3, 1, 1, 3, 5, 3, 12], [3, 5, 7, 5, 3, 3, 5, 7, 5, 3], [5, 7, 10, 7, 5, 5, 7, 10, 7, 5], [3, 5, 7, 5, 3, 3, 5, 7, 5, 3], [1, 3, 5, 3, 12, 12, 3, 5, 3, 1], [1, 3, 5, 3, 12, 12, 3, 5, 3, 1], [3, 5, 7, 5, 3, 3, 5, 7, 5, 3], [5, 7, 10, 7, 5, 5, 7, 10, 7, 5], [3, 5, 7, 5, 3, 3, 5, 7, 5, 3], [12, 3, 5, 3, 1, 1, 3, 5, 3, 12]], 'startedAtUnixTime': 0, 'turn': 0, 'tiled': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 2, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'teams': [{'teamID': 1, 'agents': [{'agentID': 1, 'x': 1, 'y': 2}, {'agentID': 2, 'x': 5, 'y': 4}, {'agentID': 3, 'x': 1, 'y': 9}], 'tilePoint': 9, 'areaPoint': 0}, {'teamID': 2, 'agents': [{'agentID': 4, 'x': 10, 'y': 2}, {'agentID': 5, 'x': 6, 'y': 7}, {'agent

In [14]:
def get_max_sel(nodes):
    m=0
    for j in range(9):
        if nodes[m]["selected"]<nodes[j]["selected"]:
            m=j
    return m

In [13]:
def reverse(tiled):
    for i in range(len(tiled)):
        for j in range(len(tiled[0])):
            if tiled[i][j]==1:
                tiled[i][j]=2
            elif tiled[i][j]==2:
                tiled[i][j]=1

In [12]:
def get_act():
    act={
        "agentID":0,
        "type":None,
        "dx":0,
        "dy":0
    }
    return act

In [11]:
def get_node(score,act):
    """
    node=dict(isTrue=True,action=act,time_win=0,score_e=score,score_a=0,per_win=0,selected=0,tiled_acted=None,teams=None,child=[])
    """
    node={
        "isTrue":True,
        "action": act,
        "time_win":0,
        "score_e":score,#敵のスコア
        "score_a":0,#行動後の自分のスコア
        "per_win":0,
        "selected":0,
        "tiled_acted":None,#前のタイルの行動後の状態(子にとってみると現在の状態)
        "teams":None,#敵の行動後の状態（現在状態）
        "child":[]
    }
    return node

In [50]:
def only_move_AI(imp,num_PLAY,data,price,move_agent_num,num_agent,nodes,exe_turn):#agent_numは味方想定 
    from copy import deepcopy
    depth=move_agent_num
    num_sel=0
    t_t=1#turn type 1なら味方 2敵
    for i in range(num_PLAY):
        if i == 0:
            if move_agent_num==0:
                for n in range(9):
                    nodes.append(get_node(data["teams"][0]["tilePoint"]+data["teams"][0]["areaPoint"],n+1))
                    nodes[n]["tiled_acted"]=deepcopy(data["tiled"])
                    agent=deepcopy(data["teams"][0]["agents"][0]) 
                    culsco(data["points"],n+1,nodes[n],agent)
            else:
                for n in range(9):
                    num_sel+=nodes[n]["selected"]
        n=get_max_ucb(nodes,num_sel,imp)
        nodes[n]["selected"]+=1
        num_sel+=1
        if nodes[n]["selected"]>=price:
            if nodes[n]["selected"]==price:
                for s in range(1,10):
                    nodes[n]["child"].append(get_node(nodes[n]["score_a"],s))
                nodes[n]["child"][0]["tiled_acted"] = deepcopy(data["tiled"]) 
                nodes[n]["child"][0]["teams"] = deepcopy(data["teams"])
                move_pos_act(nodes[n]["child"][0]["teams"][0]["agents"][move_agent_num],n+1,data["width"],data["height"],nodes[n]["child"][0]["tiled_acted"])
                reverse(nodes[n]["child"][0]["tiled_acted"])
                for m in range(1,9):
                    nodes[n]["child"][m]["teams"] = deepcopy(nodes[n]["child"][0]["teams"])
                    nodes[n]["child"][m]["tiled_acted"] = deepcopy(nodes[n]["child"][0]["tiled_acted"])
            else:
                winner=exe_enemy_node(data["points"],imp,price,nodes[n]["child"],nodes[n]["selected"]-price,num_agent,move_agent_num,0,exe_turn)
        nodes[n]["time_win"]+=get_winner(nodes[n])
        nodes[n]["per_win"]=nodes[n]["time_win"]/nodes[n]["selected"]

In [22]:
def exe_enemy_node(points,imp,price,nodes,num_sel,num_agent,move_agent_num,turn_exe,exe_turn):
    from copy import deepcopy
    if num_sel==1:
        for n in range(9):
            culsco(points,n+1,nodes[n],nodes[n]["teams"][1]["agents"][move_agent_num])
    n=get_max_ucb(nodes,num_sel,imp)
    nodes[n]["selected"]+=1
    if move_agent_num==num_agent:
        turn_exe+=1
        if turn_exe == exe_turn:
            winner=get_winner(nodes[n])
            nodes[n]["time_win"]+=winner
            nodes[n]["per_win"]=nodes[n]["time_win"]/nodes[n]["selected"]
            return 1.0-winner
            #勝率計算
    if nodes[n]["selected"]>=price:
        if nodes[n]["selected"]==price:
            winner=get_winner(nodes[n])
            for s in range(1,10):
                nodes[n]["child"].append(get_node(nodes[n]["score_a"],s))
            nodes[n]["child"][0]["tiled_acted"] = deepcopy(nodes[n]["tiled_acted"]) 
            nodes[n]["child"][0]["teams"] = deepcopy(nodes[n]["teams"])
            move_pos_act(nodes[n]["child"][0]["teams"][1]["agents"][move_agent_num],n+1,len(nodes[n]["tiled_acted"][0]),len(nodes[n]["tiled_acted"]),nodes[n]["child"][0]["tiled_acted"])
            reverse(nodes[n]["child"][0]["tiled_acted"])
            for m in range(1,9):
                    nodes[n]["child"][m]["teams"] = deepcopy(nodes[n]["child"][0]["teams"])
                    nodes[n]["child"][m]["tiled_acted"] = deepcopy(nodes[n]["child"][0]["tiled_acted"])
        else:
            winner=exe_side_node(points,imp,price,nodes[n]["child"],nodes[n]["selected"]-price,num_agent,(move_agent_num+1)%num_agent,turn_exe,exe_turn)
    else:             
        winner=get_winner(nodes[n])
    nodes[n]["time_win"]+=winner
    nodes[n]["per_win"]=nodes[n]["time_win"]/nodes[n]["selected"]
    return 1.0-winner

In [21]:
def exe_side_node(points,imp,price,nodes,num_sel,num_agent,move_agent_num,turn_exe,exe_turn): 
    from copy import deepcopy
    if num_sel==1:
        for n in range(9):
            culsco(points,n+1,nodes[n],nodes[n]["teams"][0]["agents"][move_agent_num])
    n=get_max_ucb(nodes,num_sel,imp)
    nodes[n]["selected"]+=1
    if nodes[n]["selected"]>=price:
        if nodes[n]["selected"]==price:
            winner=get_winner(nodes[n])
            for s in range(1,10):
                nodes[n]["child"].append(get_node(nodes[n]["score_a"],s))
            nodes[n]["child"][0]["tiled_acted"] = deepcopy(nodes[n]["tiled_acted"]) 
            nodes[n]["child"][0]["teams"] = deepcopy(nodes[n]["teams"])
            move_pos_act(nodes[n]["child"][0]["teams"][0]["agents"][move_agent_num],n+1,len(nodes[n]["tiled_acted"][0]),len(nodes[n]["tiled_acted"]),nodes[n]["child"][0]["tiled_acted"])
            reverse(nodes[n]["child"][0]["tiled_acted"])
            for m in range(1,9):
                    nodes[n]["child"][m]["teams"] = deepcopy(nodes[n]["child"][0]["teams"])
                    nodes[n]["child"][m]["tiled_acted"] = deepcopy(nodes[n]["child"][0]["tiled_acted"])
        else:
            winner=exe_enemy_node(points,imp,price,nodes[n]["child"],nodes[n]["selected"]-price,num_agent,move_agent_num,turn_exe,exe_turn)
    else:
        winner=get_winner(nodes[n])
    nodes[n]["time_win"]+=winner
    nodes[n]["per_win"]=nodes[n]["time_win"]/nodes[n]["selected"]
    return 1.0-winner

In [6]:
def get_winner(node):
    if node["score_e"]>node["score_a"]:
        return 0
    elif node["score_e"]<node["score_a"]:
        return 1
    else:
        return 0.5

In [7]:
def get_max_ucb(nodes,num_sel,imp):#最もUCBが大きいものの位置を返す
    from numpy import argmax 
    pos_ucb=0
    ucbs=[]
    for l in range(9):
        ucbs.append(get_UCB(imp,num_sel,nodes[l]["selected"],nodes[l]["per_win"],nodes[l]["isTrue"]))
    pos_ucb = argmax(ucbs)
    return pos_ucb

In [5]:
def get_UCB(imp,num_sel,selected,Q,isTrue):#score1　獲得スコア　UCBの計算 Q　勝率
    from math import sqrt
    from math import log
    if isTrue == False:
        return -10000
    if selected==0:
        return 10000
    return Q+imp*sqrt(2*log(num_sel)/selected)

In [4]:
def move_pos_act(agent,act,width,height,tiled):
    import math
    act=act-1
    m_x=(act%3)-1
    m_y=int(math.floor(act/3))-1
    if agent["x"]+m_x<=0 or agent["x"]+m_x>width or agent["y"]+m_y<=0 or agent["y"]+m_y>height:
        return False
    if tiled[agent["y"]+m_y-1][agent["x"]+m_x-1]==2:
        return False
    if tiled[agent["y"]+m_y-1][agent["x"]+m_x-1]==0:
        tiled[agent["y"]+m_y-1][agent["x"]+m_x-1]=1
    agent["x"]=agent["x"]+m_x
    agent["y"]=agent["y"]+m_y
    return True

In [27]:
def culsco(points,act,node,agent):
    from copy import deepcopy
    tile=deepcopy(node["tiled_acted"])
    node["isTrue"] = move_pos_act(agent,act,len(tile),len(tile[0]),tile)
    if node["isTrue"]==True:
        tl=cir2(tile,len(tile[0]),len(tile))
        score=0
        for i in range(len(tile)):
            for j in range(len(tile[0])):
                if tl[i][j]==4:
                    score+=abs(points[i][j])
                if tl[i][j]==9:
                    score+=points[i][j]
        node["score_a"]=score

In [2]:
def cir2(tiled,width,height):#囲み判定
    c=[]
    for i in range(height):
        o=[]
        for n in range(width):
            o.append(0)
        c.append(o)
    for i in range(height):
        for n in range(width):
            if tiled[i][n]==1:
                c[i][n]=9
                for k in range(width):
                    if k+1>n:
                        break
                    elif tiled[i][n-k-1]==1:
                        break
                    c[i][n-k-1]+=1
                for k in range(width):
                    if n+k+1>=width:
                        break
                    elif tiled[i][n+k+1]==1:
                        break
                    c[i][n+k+1]+=1
                for k in range(height):
                    if k+1>i:
                        break
                    elif tiled[i-k-1][n]==1:
                        break
                    c[i-k-1][n]+=1
                for k in range(height):
                    if i+k+1>=height:
                        break
                    elif tiled[i+k+1][n]==1:
                        break
                    c[i+k+1][n]+=1
    if not c[height-1][width-1]==9:
        c[height-1][width-1]=0
    if not c[height-1][0]==9:
        c[height-1][0]=0
    if not c[0][width-1]==9:
        c[0][width-1]=0
    if not c[0][0]==9:
        c[0][0]=0
    for s in range(width*height):
        count_exe=0
        for n in range(height):
            for i in range(width):
                if c[n][i]<4:
                    if n-1>=0:
                        if 0<c[n-1][i]<9:
                            c[n-1][i]=0
                            count_exe+=1
                    if i-1>=0:
                        if 0<c[n][i-1]<9:
                            c[n][i-1]=0
                            count_exe+=1
                    if n+1<len(c):
                        if 0<c[n+1][i]<9:
                            c[n+1][i]=0
                            count_exe+=1
                    if i+1<len(c[n]):
                        if 0<c[n][i+1]<9:
                            c[n][i+1]=0
                            count_exe+=1
        if count_exe==0:#1回も実行されなかったら終わる
            break
    return c

In [45]:
import json
f=open('A-1.json')
data=json.load(f)
print(data["teams"][0])
for i in range(9):
    print(data["tiled"][i])
for i in range(3):
    print(data["teams"][0]["agents"][i]["y"]-1,data["teams"][0]["agents"][i]["x"]-1)
    print(data["tiled"][data["teams"][0]["agents"][i]["y"]-1][data["teams"][0]["agents"][i]["x"]-1])
S=move_pos_act(data["teams"][0]["agents"][0],1,len(tiled[0]),len(tiled),data["tiled"])
print(S)

{'teamID': 1, 'agents': [{'agentID': 1, 'x': 1, 'y': 2}, {'agentID': 2, 'x': 5, 'y': 4}, {'agentID': 3, 'x': 1, 'y': 9}], 'tilePoint': 9, 'areaPoint': 0}
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 2, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 2]
1 0
1
3 4
1
8 0
1
False
